##### L44 v1.4: cleans up the code, deleting lesson progression. See L43 v1.3 file to see lesson progression.
-----------
##### Define Variables

In [1]:
pair = "EUR_USD"
gran1 = "M1"
gran2 = "S30"

# Only used to load the candle file (after modified candle savefile-name to incl date).
date = "2021-05-27"
pkl_01_name = f"insidebar_{date}_{pair}_{gran1}_rawdata"

# Only used to calculate the risk_reward_ratio for def process_buy and def process_sell. Must be the same as inside bar explore.
SLOSS = 0.4
TPROFIT = 1.2
ENTRY_PRC = 0.1
risk_reward_ratio = TPROFIT / SLOSS

-----------
##### Begin Code

In [2]:
import pandas as pd
import A2_utils as utils
import datetime as dt
from dateutil.parser import *

# import pickle as pkl
# import plotly.graph_objects as go

In [3]:
# Read trade info from gran1
df_trades = pd.read_pickle(f"results/{pkl_01_name}.pkl")
print(f"{gran1} df columns, rows: {df_trades.shape}")
print("")
df_trades.head(3)

M1 df columns, rows: (153, 17)



,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
4,2021-05-26T15:05:00.000000000Z,1.22211,1.22212,1.22196,1.22197,1.22191,1.22203,0.00016,1.22213,1.22192,0.00021,-1,1,1,1.222151,1.222067,1.222403
18,2021-05-26T15:19:00.000000000Z,1.22132,1.22141,1.22128,1.22141,1.22135,1.22147,0.00013,1.22151,1.22124,0.00027,1,-1,-1,1.221213,1.221321,1.220889
34,2021-05-26T15:35:00.000000000Z,1.22048,1.22048,1.22014,1.22014,1.22008,1.22020,0.00034,1.22060,1.22012,0.00048,-1,1,1,1.220648,1.220456,1.221224


In [4]:
# Read candles from gran2
df_raw = pd.read_pickle(utils.get_his_data_filename_withdate(date, pair, gran2))
print(f"{gran2} df columns, rows: {df_raw.shape}")
print("")
df_raw.head(3)

S30 df columns, rows: (2746, 14)



,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-05-26T15:00:00.000000000Z,248,1.22169,1.22194,1.22169,1.22184,1.22163,1.22188,1.22162,1.22178,1.22175,1.22201,1.22175,1.22190
1,2021-05-26T15:00:30.000000000Z,146,1.22183,1.22186,1.22164,1.22164,1.22177,1.22180,1.22157,1.22157,1.22189,1.22192,1.22170,1.22170
2,2021-05-26T15:01:00.000000000Z,203,1.22162,1.22168,1.22144,1.22146,1.22156,1.22162,1.22137,1.22139,1.22169,1.22175,1.22150,1.22152


In [5]:
# Convert price columns in df_raw (gran2) to numeric
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [6]:
# Convert time column values from string to datetime in gran1 and gran2
df_trades['time'] = [parse(x) for x in df_trades['time']]
df_raw['time'] = [parse(x) for x in df_raw['time']]

# Create new columns in df_trades (gran1)
df_trades['next'] = df_trades['time'].shift(-1)
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=0, minutes=0, seconds=30)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=0, minutes=1, seconds=0) #L42 Addition

# Explanation from BF Software:
# time - Start of current candle time
# next - Start of next candle time
# trade_end - the last candle the trade can run to (end of next candle period minus gran2 candle)
# trade_start - the next gran2 after start of current candle trade starts

# Check and make sure new columns are in place:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
4,2021-05-26 15:05:00+00:00,2021-05-26 15:19:00+00:00,2021-05-26 15:19:30+00:00,2021-05-26 15:06:00+00:00
18,2021-05-26 15:19:00+00:00,2021-05-26 15:35:00+00:00,2021-05-26 15:35:30+00:00,2021-05-26 15:20:00+00:00
34,2021-05-26 15:35:00+00:00,2021-05-26 15:38:00+00:00,2021-05-26 15:38:30+00:00,2021-05-26 15:36:00+00:00
37,2021-05-26 15:38:00+00:00,2021-05-26 15:54:00+00:00,2021-05-26 15:54:30+00:00,2021-05-26 15:39:00+00:00
53,2021-05-26 15:54:00+00:00,2021-05-26 16:00:00+00:00,2021-05-26 16:00:30+00:00,2021-05-26 15:55:00+00:00


In [7]:
# L41 Reset the index and drop na valurs in df_trades
df_trades.dropna(inplace = True)
df_trades.reset_index(drop = True, inplace = True)
df_trades[['time', 'next', 'trade_end']].head()

,time,next,trade_end
0,2021-05-26 15:05:00+00:00,2021-05-26 15:19:00+00:00,2021-05-26 15:19:30+00:00
1,2021-05-26 15:19:00+00:00,2021-05-26 15:35:00+00:00,2021-05-26 15:35:30+00:00
2,2021-05-26 15:35:00+00:00,2021-05-26 15:38:00+00:00,2021-05-26 15:38:30+00:00
3,2021-05-26 15:38:00+00:00,2021-05-26 15:54:00+00:00,2021-05-26 15:54:30+00:00
4,2021-05-26 15:54:00+00:00,2021-05-26 16:00:00+00:00,2021-05-26 16:00:30+00:00


In [17]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

# ------------------------
# New L43 Function (00:27)

def end_hit_calc(direction, SL, current_price, entry_price):
    delta = current_price - entry_price
    full_delta = entry_price - SL
    fraction = abs(delta / full_delta)
    print(f"   func_end_hit_calc: current_price:{current_price:.5f}, entry_price:{entry_price:.5f}, delta (current-entry, pips):{delta*10000:.2f}, full_delta (entry-SL, pips):{full_delta*10000:.2f}, fraction(abs):{fraction:.2f}")
   
    if direction == 1 and current_price >= entry_price:
        return fraction
    elif direction == 1 and current_price < entry_price:
        return -fraction
    elif direction == -1 and current_price <= entry_price:
        return fraction
    elif direction == -1 and current_price > entry_price:
        return -fraction
    print ("ERROR")

# Modified L42 functions

def process_buy(start_index, TP, SL, prices, entry_price):
    for current_price in prices[start_index:]:
        if current_price >= TP:
            return risk_reward_ratio
        elif current_price <= SL:
            return -1.0
    fraction = end_hit_calc(1, SL, current_price, entry_price)
    print(f"   func_process_buy: fraction: {fraction:.2f}, current_price: {current_price}, entry_price: {entry_price}")
    return fraction

def process_sell(start_index, TP, SL, prices, entry_price):
    for current_price in prices[start_index:]:
        if current_price <= TP:
            return risk_reward_ratio
        elif current_price >= SL:
            return -1.0
    fraction = end_hit_calc(-1, SL, current_price, entry_price)
    print(f"   func_process_sell: fraction: {fraction:.2f}, current_price: {current_price}, entry_price: {entry_price}")
    return fraction

def process_trade(start_index, direction, TP, SL, prices, entry_price): # Asw modified start_price to entry_price
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, entry_price)
    elif direction == -1:
        return process_sell(start_index, TP, SL, prices, entry_price)

# L42-B (06:26)
def process_gran2(gran2_df, row):
    result = 0.0
    for index, price in enumerate(gran2_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            print(f"   func_process_gran2: Trigger {signal_text(row.SIGNAL)} at index {index} and price: {price:.5f} based on entry signal: {row.ENTRY:.5f}") # Asw Mod
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, gran2_df.mid_c.values, row.ENTRY)
            break
    return result

In [18]:
# For every buy signal (gran1), how many times (gran2) do we get to analyse the data for trade trigger (L41-A)
# The x value in results in shape (x,y) after date time signal price = the number of gran2 signals are analysed
# Line process_gran2 = show trade trigger point from the buy signal (if triggered) (L41-B)

# L42-B (06:46)
total = 0
for index, row in df_trades.iterrows():
    fine_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)] #L42
    print(f"{row.time} {signal_text(row.SIGNAL)} signal at {row.ENTRY:.5f} {fine_data.shape}")
    total = total + process_gran2(fine_data, row)
    print("")
print(f"\nTotal R Result: {total}")

2021-05-26 15:05:00+00:00 BUY signal at 1.22215 (28, 14)

2021-05-26 15:19:00+00:00 SELL signal at 1.22121 (32, 14)
   func_process_gran2: Trigger SELL at index 0 and price: 1.22116 based on entry signal: 1.22121

2021-05-26 15:35:00+00:00 BUY signal at 1.22065 (6, 14)
   func_process_gran2: Trigger BUY at index 3 and price: 1.22066 based on entry signal: 1.22065
   func_end_hit_calc: current_price:1.22052, entry_price:1.22065, delta (current-entry, pips):-1.28, full_delta (entry-SL, pips):1.92, fraction(abs):0.67
   func_process_buy: fraction: -0.67, current_price: 1.22052, entry_price: 1.220648

2021-05-26 15:38:00+00:00 BUY signal at 1.22079 (32, 14)
   func_process_gran2: Trigger BUY at index 4 and price: 1.22090 based on entry signal: 1.22079

2021-05-26 15:54:00+00:00 SELL signal at 1.21969 (12, 14)

2021-05-26 16:00:00+00:00 SELL signal at 1.22027 (26, 14)
   func_process_gran2: Trigger SELL at index 10 and price: 1.22026 based on entry signal: 1.22027

2021-05-26 16:13:00+00:00